In [21]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from sklearn import linear_model, svm, tree
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score, median_absolute_error

In [3]:
target = 'Item_Outlet_Sales'
training = pd.read_csv('training.csv')
testing = pd.read_csv('testing.csv')
x_train, x_test, y_train, y_test = training.drop(columns=[target]), testing.drop(columns=[target]), training[target], testing[target]
x_train.columns

Index(['Low_Fat', 'Regular', 'Dairy', 'Soft_Drinks', 'Meat',
       'Fruits_Vegetables', 'Household', 'Baking', 'Snack', 'Fronzen_Foods',
       'Breakfast', 'Health_Hygiene', 'Hard_Drinks', 'Canned', 'Breads',
       'Starchy', 'Others', 'Seafood', 'Item_MRP', 'Outlet_Size',
       'Outlet_Location_Type', 'Outlet_Type', 'Item_Weight', 'Item_Visibility',
       'Outlet_Year', 'Item_Identifier_8', 'Item_Identifier_6',
       'Item_Identifier_7', 'Item_Identifier_3', 'Item_Identifier_5',
       'Item_Identifier_4', 'Item_Identifier_9', 'Item_Identifier_2',
       'Item_Identifier_1', 'Outlet_Freq'],
      dtype='object')

In [24]:
clr = tree.DecisionTreeRegressor().fit(x_train, y_train)
y_pred = clr.predict(x_test)

In [25]:
metric = format(mean_absolute_error(y_test, y_pred), '.2f')
metric

'1021.82'